In [1]:
import requests
from bs4 import BeautifulSoup

urls = ["https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1"]

#Action "https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1" 
#SinglePlayer https://store.steampowered.com/search/?filter=topsellers&tags=4182&supportedlang=english&ndl=1
#Adventure https://store.steampowered.com/search/?filter=topsellers&tags=21&supportedlang=english&ndl=1
#Indie https://store.steampowered.com/search/?filter=topsellers&tags=492&supportedlang=english&ndl=1
#Simulation https://store.steampowered.com/search/?filter=topsellers&tags=599&supportedlang=english&ndl=1
#Strategy https://store.steampowered.com/search/?filter=topsellers&tags=9&supportedlang=english&ndl=1
#Casual https://store.steampowered.com/search/?filter=topsellers&tags=597&supportedlang=english&ndl=1
#RPG https://store.steampowered.com/search/?filter=topsellers&tags=122&supportedlang=english&ndl=1
#Multiplayer https://store.steampowered.com/search/?filter=topsellers&tags=3859&supportedlang=english&ndl=1
#Atmospheric https://store.steampowered.com/search/?filter=topsellers&tags=4166&supportedlang=english&ndl=1
#2d https://store.steampowered.com/search/?filter=topsellers&tags=3871&supportedlang=english&ndl=1 
#story rich https://store.steampowered.com/search/?filter=topsellers&tags=1742&supportedlang=english&ndl=1
#co-op https://store.steampowered.com/search/?filter=topsellers&tags=1685&supportedlang=english&ndl=1
#first-person https://store.steampowered.com/search/?filter=topsellers&tags=3839&supportedlang=english&ndl=1
#3d https://store.steampowered.com/search/?filter=topsellers&tags=4191&supportedlang=english&ndl=1
#open world https://store.steampowered.com/search/?filter=topsellers&tags=1695&supportedlang=english&ndl=1

app_id = []

for url in urls:
    response = requests.get(url)
# , headers={
#     'cookie': 'steamCountry=KR!13817a80e36b14f1a94dd77fb2142a64; browserid=3999848730290483524; sessionid=2225f1149bbcf81b9631038f; timezoneOffset=32400,0; app_impressions=2344520@1_7_7_7000_150_1|2138330@1_7_7_7000_150_1|2358720@1_7_7_7000_150_1|2074920@1_7_7_7000_150_1|2183900@1_7_7_7000_150_1|1245620@1_7_7_7000_150_1|2139460@1_7_7_7000_150_1|1049590@1_7_7_7000_150_1|1091500@1_7_7_7000_150_1|578080@1_7_7_7000_150_1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
#     'accept-language':'ko-KR,ko;q=0.9',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-encoding': 'gzip, deflate, br, zstd',
#     'cache-control':'no-cache'
# }

    soup = BeautifulSoup(response.content, 'html.parser')

    app_ids = []

    container = soup.find('div', id='search_result_container')

    if container:
        for item in container.find_all('a'):
            link =item['href']
            if 'app' in link:
                ex = link.split('app')[1].split('/')[1]
                app_ids.append(ex)

    app_ids=app_ids[:10] # 상위 10개

    for i in range(len(app_ids)):
        app_id.append(app_ids[i])

app_id


['2246340',
 '578080',
 '2139460',
 '1049590',
 '1091500',
 '2074920',
 '1245620',
 '2358720',
 '1364780',
 '2828860']

In [2]:
import requests
import pandas as pd

# Steam Store API에서 특정 게임의 세부 정보를 가져오는 함수
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]["success"]:
            game_data = data[str(app_id)]["data"]
            return {
                "app_id": app_id,
                "name": game_data.get("name"),
                "genres": ", ".join([genre['description'] for genre in game_data.get("genres", [])]),  # 장르
                "header_image": game_data.get("header_image"),  # 헤더 이미지
                "release_date": game_data.get("release_date", {}).get("date"),  # 출시일
                "price": game_data.get("price_overview", {}).get("final_formatted", "가격 정보 없음")  # 가격 정보
            }
    return {}

# Steam API를 통해 리뷰 데이터를 가져오는 함수
def get_reviews_for_game(app_id, num_reviews=100):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        'json': 1,
        'num_per_page': num_reviews,  # 한 번에 가져올 리뷰 수
        'filter': 'recent',  # 최신 리뷰만 가져옴
        'language': 'english'  # 영어로 된 리뷰만 가져옴
    }
    response = requests.get(url, params=params)
    data = response.json()

    reviews = []

    if 'reviews' in data:
        for review in data['reviews']:
            reviews.append({
                'review_text': review.get('review'),  # 리뷰 내용
                'review_score': review.get('voted_up'),  # 추천 여부
                'language': review.get('language'),  # 리뷰 언어
                'author_steamid': review.get('author', {}).get('steamid'),  # 작성자 Steam ID
                'playtime_at_review': review.get('author', {}).get('playtime_at_review'),  # 리뷰 작성 당시의 플레이 시간
                'timestamp_created': review.get('timestamp_created')  # 리뷰 작성 시간
            })

    return reviews

# Steam ID를 사용자 닉네임으로 변환하는 함수 (API 키 사용하지 않음)
def resolve_vanity_url(steam_id):
    # API 키 없이, 닉네임을 직접 반환
    return steam_id  # 원래 Steam ID 반환

# 게임 이름, 장르, 이미지, 리뷰 정보 등을 포함한 데이터를 DataFrame으로 정리하는 함수
def get_game_data_with_reviews(app_ids, num_reviews=100):
    all_data = []

    for app_id in app_ids:
        # 1. 게임 세부 정보 가져오기
        game_info = get_game_details(app_id)
        
        if game_info:
            # 2. 게임에 대한 리뷰 가져오기
            reviews = get_reviews_for_game(app_id, num_reviews)
            
            # 3. 각 리뷰에 게임 세부 정보 추가
            for review in reviews:
                # 4. Steam ID를 사용자 닉네임으로 변환 (닉네임으로 처리됨)
                author_nickname = resolve_vanity_url(review['author_steamid'])
                
                all_data.append({
                    'app_id': game_info['app_id'],  # 게임 ID
                    'game_name': game_info['name'],  # 게임 이름
                    'game_genres': game_info['genres'],  # 게임 장르
                    'game_image': game_info['header_image'],  # 게임 이미지
                    'game_release_date': game_info['release_date'],  # 출시일
                    'game_price': game_info['price'],  # 가격
                    'review_text': review['review_text'],  # 리뷰 내용
                    'review_score': review['review_score'],  # 리뷰 평가
                    'language': review['language'],  # 리뷰 언어
                    'author': author_nickname,  # 작성자 닉네임
                    'playtime_at_review': review['playtime_at_review'],  # 작성 당시 플레이 시간
                    'timestamp_created': review['timestamp_created']  # 리뷰 작성 시간
                })

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_data)
    return df

# 게임 데이터 및 리뷰 가져오기
df_game_data = get_game_data_with_reviews(app_id, num_reviews=50)  # 각 게임당 50개의 리뷰 가져오기

# CSV 파일로 저장
df_game_data.to_csv('Action.csv', index=False)


In [3]:
Example=pd.read_csv('Action.csv')
Example

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"its a learning curve, but worth it. hands down...",True,english,76561198150855629,1597,1727240369
1,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,good,True,english,76561199006157895,3008,1727235379
2,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"My first game, I was smacked in the head with ...",True,english,76561198856507936,155,1727233525
3,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,The OG BR for those who enjoy a challenge.,True,english,76561197961032885,30354,1727227781
4,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,everyone but me is hacking buts its aight,True,english,76561198007144841,30199,1727226565
...,...,...,...,...,...,...,...,...,...,...,...,...
314,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",This game is so fucking fun.\r\n\r\neverything...,True,english,76561199013194354,139,1727238221
315,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I’ve played 2 missions so far and I’m hooked. ...,True,english,76561198990587616,47,1727238188
316,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",fuck the water mechanic,False,english,76561198105737823,27,1727238179
317,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I'm going to keep this short and sweet. The ga...,False,english,76561198026328144,48,1727238109


In [4]:
Example['app_id'].value_counts()

app_id
578080     50
1091500    50
1245620    50
2358720    50
1364780    50
2828860    50
1049590    19
Name: count, dtype: int64

In [5]:
Example1= Example.copy()
Example1

,app_id,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"its a learning curve, but worth it. hands down...",True,english,76561198150855629,1597,1727240369
1,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,good,True,english,76561199006157895,3008,1727235379
2,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"My first game, I was smacked in the head with ...",True,english,76561198856507936,155,1727233525
3,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,The OG BR for those who enjoy a challenge.,True,english,76561197961032885,30354,1727227781
4,578080,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,everyone but me is hacking buts its aight,True,english,76561198007144841,30199,1727226565
...,...,...,...,...,...,...,...,...,...,...,...,...
314,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",This game is so fucking fun.\r\n\r\neverything...,True,english,76561199013194354,139,1727238221
315,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I’ve played 2 missions so far and I’m hooked. ...,True,english,76561198990587616,47,1727238188
316,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",fuck the water mechanic,False,english,76561198105737823,27,1727238179
317,2828860,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I'm going to keep this short and sweet. The ga...,False,english,76561198026328144,48,1727238109


In [6]:
Example1=Example1.drop(['app_id', ], axis=1) # app_id 삭제
Example1 = Example1[Example1['language'] == 'english'] # english 
Example1 = Example1.reset_index(drop=True) # 재정렬
Example1 

,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"its a learning curve, but worth it. hands down...",True,english,76561198150855629,1597,1727240369
1,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,good,True,english,76561199006157895,3008,1727235379
2,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,"My first game, I was smacked in the head with ...",True,english,76561198856507936,155,1727233525
3,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,The OG BR for those who enjoy a challenge.,True,english,76561197961032885,30354,1727227781
4,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,"Dec 21, 2017",가격 정보 없음,everyone but me is hacking buts its aight,True,english,76561198007144841,30199,1727226565
...,...,...,...,...,...,...,...,...,...,...,...
314,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",This game is so fucking fun.\r\n\r\neverything...,True,english,76561199013194354,139,1727238221
315,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I’ve played 2 missions so far and I’m hooked. ...,True,english,76561198990587616,47,1727238188
316,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",fuck the water mechanic,False,english,76561198105737823,27,1727238179
317,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,"24 Sep, 2024","₩ 28,800",I'm going to keep this short and sweet. The ga...,False,english,76561198026328144,48,1727238109


In [7]:
# Example1 전처리
Example1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   game_name           319 non-null    object
 1   game_genres         319 non-null    object
 2   game_image          319 non-null    object
 3   game_release_date   319 non-null    object
 4   game_price          319 non-null    object
 5   review_text         318 non-null    object
 6   review_score        319 non-null    bool  
 7   language            319 non-null    object
 8   author              319 non-null    int64 
 9   playtime_at_review  319 non-null    int64 
 10  timestamp_created   319 non-null    int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 25.4+ KB


In [8]:
Example1['game_name'].isnull()
Example1['game_name'].value_counts() # 개수확인  총합 .sum()
Example1['game_name'].isnull() # null 확인 총합 .sum()

0      False
1      False
2      False
3      False
4      False
       ...  
314    False
315    False
316    False
317    False
318    False
Name: game_name, Length: 319, dtype: bool

In [9]:
Example1['game_genres'].value_counts().sum()
Example1['game_genres'].isnull().sum()

0

In [10]:
Example1['game_image'].value_counts().sum()
Example1['game_image'].isnull().sum()



0

In [11]:
Example1['language'].value_counts().sum()
Example1['language'].isnull().sum()

0

In [12]:
Example1['game_release_date'].value_counts().sum()
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].value_counts()


# 이탈리아어 월 이름을 영어로 변환하는 사전
month_map = {
    'gen.': 'Jan',
    'feb.': 'Feb',
    'mar.': 'Mar',
    'apr.': 'Apr',
    'mag.': 'May',
    'giu.': 'Jun',
    'lug.': 'Jul',
    'ago.': 'Aug',
    'set.': 'Sep',
    'ott.': 'Oct',
    'nov.': 'Nov',
    'dic.': 'Dec'
}

# 날짜 변환 함수 정의
def parse_date(date_str):
    # 이탈리아어 월 이름을 영어로 변환
    for italian_month, english_month in month_map.items():
        if italian_month in date_str:
            date_str = date_str.replace(italian_month, english_month)

    # 여러 가지 날짜 형식 처리
    try:
        # 'Sep 19, 2024' 형식
        return pd.to_datetime(date_str, format='%b %d, %Y')
    except ValueError:
        pass
    try:
        # '19 Sep, 2024' 형식
        return pd.to_datetime(date_str, format='%d %b, %Y')
    except ValueError:
        pass
    try:
        # '9/set./2024' 형식 (이탈리아어 월 변환 후 처리)
        return pd.to_datetime(date_str, format='%d/%b/%Y')
    except ValueError:
        pass
    
    return pd.NaT  # 변환 실패 시 NaT 반환

# 데이터프레임의 날짜 열 변환
Example1['game_release_date'] = Example1['game_release_date'].apply(parse_date)
Example1['game_release_date'].isnull().sum()
Example1['game_release_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 319 entries, 0 to 318
Series name: game_release_date
Non-Null Count  Dtype         
--------------  -----         
319 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.6 KB


In [13]:
Example1['review_score']
Example1['review_score'].isnull().sum()
Example1['review_score'].value_counts()
Example1['review_score'].value_counts().sum()

319

In [14]:
Example1['review_text'].value_counts().sum()
Example1['review_text'].value_counts()
Example1['review_text'].isnull().sum()
Example1['review_text'].isnull()
Example1 = Example1.dropna(subset=['review_text'], axis=0)

In [15]:
# game_price
Example1['game_price'].value_counts().sum()

318

In [16]:
Example1['game_price'].isnull().sum()


0

In [17]:
Example1['game_price'].value_counts()

game_price
가격 정보 없음      68
CDN$ 39.99    50
₩ 45,360      50
₩ 64,800      50
₩ 34,900      50
₩ 28,800      50
Name: count, dtype: int64

In [18]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') # 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

C:\Users\user\AppData\Local\Temp\ipykernel_12632\107871597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') #
C:\Users\user\AppData\Local\Temp\ipykernel_12632\107871597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')


In [19]:
# 특정 값 변경
Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€') 

Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')

C:\Users\user\AppData\Local\Temp\ipykernel_12632\4294488823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('44,99€', '44.99€')
C:\Users\user\AppData\Local\Temp\ipykernel_12632\4294488823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('R$ 199,90', 'R$ 199.90')


In [20]:
import re

# 통화 기호에 따라 숫자를 변환하는 함수
def extract_numbers(price_str):
    if isinstance(price_str, str):  # price_str이 문자열인지 확인
        # 통화 기호에 따라 환율을 설정
        if '₩' in price_str:
            multiplier = 1  # 원화: 그대로 사용
        elif 'CDN$' in price_str:
            multiplier = 950  # 캐나다 달러 기준, 예: 1 CAD = 950 KRW
        elif '$' in price_str:
            multiplier = 1300  # 미국 달러 기준, 예: 1 USD = 1300 KRW
        elif '€' in price_str:
            multiplier = 1400  # 유로 기준, 예: 1 EUR = 1400 KRW
        elif '¥' in price_str:
            multiplier = 10  # 엔화 기준, 예: 1 JPY = 10 KRW
        elif 'R$' in price_str:
            multiplier = 300  # 브라질 헤알 기준, 예: 1 BRL = 300 KRW
        else:
            multiplier = 1  # 통화 기호가 없는 경우 그대로 사용

        # 소수점이 포함된 숫자만 추출 (쉼표와 공백 제거)
        numbers = re.sub(r'[^\d.]', '', price_str)
        
        # 추출한 숫자 문자열을 소수로 변환한 후 환율 적용
        return float(numbers) * multiplier if numbers else 0  # 숫자에 통화 단위 곱하기
    return 0  # 문자열이 아닐 경우 0 반환

# 'game_price' 열 변환
Example1['game_price'] = Example1['game_price'].replace('가격 정보 없음', '0')  # 가격 정보가 없는 경우 0으로 대체
Example1['game_price'] = Example1['game_price'].apply(extract_numbers)  # 각 가격 문자열에 대해 함수 적용

C:\Users\user\AppData\Local\Temp\ipykernel_12632\1318368436.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].replace('가격 정보 없음', '0')  # 가격 정보가 없는 경우 0으로 대체
C:\Users\user\AppData\Local\Temp\ipykernel_12632\1318368436.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['game_price'] = Example1['game_price'].apply(extract_numbers)  # 각 가격 문자열에 대해 함수 적용


In [21]:
Example1['game_price'].value_counts()


game_price
0.0        68
37990.5    50
45360.0    50
64800.0    50
34900.0    50
28800.0    50
Name: count, dtype: int64

In [22]:
Example1['game_price'].isnull().sum()


0

In [23]:
Example1['game_price'].astype(int)

0          0
1          0
2          0
3          0
4          0
       ...  
314    28800
315    28800
316    28800
317    28800
318    28800
Name: game_price, Length: 318, dtype: int32

In [24]:
Example1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 318 entries, 0 to 318
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   game_name           318 non-null    object        
 1   game_genres         318 non-null    object        
 2   game_image          318 non-null    object        
 3   game_release_date   318 non-null    datetime64[ns]
 4   game_price          318 non-null    float64       
 5   review_text         318 non-null    object        
 6   review_score        318 non-null    bool          
 7   language            318 non-null    object        
 8   author              318 non-null    int64         
 9   playtime_at_review  318 non-null    int64         
 10  timestamp_created   318 non-null    int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 27.6+ KB


In [25]:
# review_text 영어만 인식한것만
def is_english(text):
    # 문자열이 None이 아닐 때 영어 문자가 포함되어 있는지 확인
    if isinstance(text, str):
        return bool(re.match(r'^[\x00-\x7F]*$', text))
    return False

# 영어가 아닌 행의 인덱스 찾기 (조건 반전)
indices_to_drop = Example1[~Example1['review_text'].apply(is_english)].index

# 기존 데이터프레임에서 영어가 아닌 행 삭제
Example1.drop(indices_to_drop, inplace=True)

# 결과 출력
Example1

C:\Users\user\AppData\Local\Temp\ipykernel_12632\1213455055.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1.drop(indices_to_drop, inplace=True)


,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,"its a learning curve, but worth it. hands down...",True,english,76561198150855629,1597,1727240369
1,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,good,True,english,76561199006157895,3008,1727235379
2,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,"My first game, I was smacked in the head with ...",True,english,76561198856507936,155,1727233525
3,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,The OG BR for those who enjoy a challenge.,True,english,76561197961032885,30354,1727227781
4,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,everyone but me is hacking buts its aight,True,english,76561198007144841,30199,1727226565
...,...,...,...,...,...,...,...,...,...,...,...
313,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,"Great concept, early phase of execution. Give ...",True,english,76561198050609088,89,1727238232
314,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,This game is so fucking fun.\r\n\r\neverything...,True,english,76561199013194354,139,1727238221
316,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,fuck the water mechanic,False,english,76561198105737823,27,1727238179
317,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,I'm going to keep this short and sweet. The ga...,False,english,76561198026328144,48,1727238109


In [26]:
#댓글 날짜, 닉네임, 사용자 플레이 시간 전처리

# 댓글 날짜 
Example1['playtime_at_review'].value_counts()
Example1['playtime_at_review'].isnull().sum() # 417
Example1['playtime_at_review'].value_counts().sum() # 6581
Example1['playtime_at_review']=Example1['playtime_at_review']/60
Example1['playtime_at_review']=Example1['playtime_at_review'].fillna(0)
Example1['playtime_at_review'].isnull().sum()
Example1['playtime_at_review'].astype(int)


# 사용자 플레이 시간 
Example1['timestamp_created'].isnull().sum() # 0
Example1['timestamp_created'].value_counts().sum() # 6581
#  Unix 타임스탬프
import pandas as pd

Example1['timestamp_created']
Example1['timestamp_created']=pd.to_datetime(Example1['timestamp_created'], unit='s')




C:\Users\user\AppData\Local\Temp\ipykernel_12632\2510639514.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['playtime_at_review']=Example1['playtime_at_review']/60
C:\Users\user\AppData\Local\Temp\ipykernel_12632\2510639514.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Example1['playtime_at_review']=Example1['playtime_at_review'].fillna(0)
C:\Users\user\AppData\Local\Temp\ipykernel_12632\2510639514.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [27]:
Example1['game_name']

0      PUBG: BATTLEGROUNDS
1      PUBG: BATTLEGROUNDS
2      PUBG: BATTLEGROUNDS
3      PUBG: BATTLEGROUNDS
4      PUBG: BATTLEGROUNDS
              ...         
313     The Forever Winter
314     The Forever Winter
316     The Forever Winter
317     The Forever Winter
318     The Forever Winter
Name: game_name, Length: 302, dtype: object

In [28]:
Example1.to_csv('Action.csv')

In [29]:
ex=pd.read_csv('Action.csv')
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          302 non-null    int64  
 1   game_name           302 non-null    object 
 2   game_genres         302 non-null    object 
 3   game_image          302 non-null    object 
 4   game_release_date   302 non-null    object 
 5   game_price          302 non-null    float64
 6   review_text         302 non-null    object 
 7   review_score        302 non-null    bool   
 8   language            302 non-null    object 
 9   author              302 non-null    int64  
 10  playtime_at_review  302 non-null    float64
 11  timestamp_created   302 non-null    object 
dtypes: bool(1), float64(2), int64(2), object(7)
memory usage: 26.4+ KB


In [30]:
ex

,Unnamed: 0,game_name,game_genres,game_image,game_release_date,game_price,review_text,review_score,language,author,playtime_at_review,timestamp_created
0,0,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,"its a learning curve, but worth it. hands down...",True,english,76561198150855629,26.616667,2024-09-25 04:59:29
1,1,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,good,True,english,76561199006157895,50.133333,2024-09-25 03:36:19
2,2,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,"My first game, I was smacked in the head with ...",True,english,76561198856507936,2.583333,2024-09-25 03:05:25
3,3,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,The OG BR for those who enjoy a challenge.,True,english,76561197961032885,505.900000,2024-09-25 01:29:41
4,4,PUBG: BATTLEGROUNDS,"Action, Adventure, Massively Multiplayer, Free...",https://shared.akamai.steamstatic.com/store_it...,2017-12-21,0.0,everyone but me is hacking buts its aight,True,english,76561198007144841,503.316667,2024-09-25 01:09:25
...,...,...,...,...,...,...,...,...,...,...,...,...
297,313,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,"Great concept, early phase of execution. Give ...",True,english,76561198050609088,1.483333,2024-09-25 04:23:52
298,314,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,This game is so fucking fun.\r\n\r\neverything...,True,english,76561199013194354,2.316667,2024-09-25 04:23:41
299,316,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,fuck the water mechanic,False,english,76561198105737823,0.450000,2024-09-25 04:22:59
300,317,The Forever Winter,"Action, Early Access",https://shared.akamai.steamstatic.com/store_it...,2024-09-24,28800.0,I'm going to keep this short and sweet. The ga...,False,english,76561198026328144,0.800000,2024-09-25 04:21:49
